### Read Table

In [0]:
from pyspark.sql import functions as F

fact_orders_df = spark.table("olist_ecommerce.gold.fact_orders")

### Define reference date

In [0]:
reference_date = fact_orders_df.select(
    F.max("purchase_timestamp")).collect()[0][0]

### Build RFM table

In [0]:
rfm_df = (
    fact_orders_df
    .groupBy("customer_unique_id")
    .agg(
        F.datediff(F.lit(reference_date),F.max("purchase_timestamp")).alias("recency"),
        F.countDistinct("order_id").alias("frequency"),
        F.sum("order_value").alias("monetary")
    )
)

### Save RFM features

In [0]:
rfm_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("olist_ecommerce.gold.customer_rfm")